**indicNER**

**INSTALLING REQUIREMENTS**

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=03c9b5f413459ba5ff0f014f8aea3e6b8683eb733157a75390bb5c304d6ce48c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


**DOWNLOADING DATASET**

In [2]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
naamapadam = load_dataset('ai4bharat/naamapadam', 'hi')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
naamapadam

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [22]:
naamapadam.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [3]:
column_names = naamapadam["train"].column_names
print(column_names)

features = naamapadam["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [23]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [24]:

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


**TOKENIZER FUNCTION**

In [25]:
padding = "max_length"
def tokenize_and_align_labels_custom(input_data):
    tokenized_data = tokenizer(
        input_data[text_column_name],
        padding_strategy=padding,
        truncation=True,
        max_length=512,
        is_split_into_words=True,
    )
    aligned_labels = []

    for index, label_seq in enumerate(input_data[label_column_name]):
        word_ids = tokenized_data.word_ids(batch_index=index)
        prev_word_index = None
        seq_labels = []

        for word_index in word_ids:
            if word_index is None:
                seq_labels.append(-100)
            elif word_index != prev_word_index:
                seq_labels.append(label_seq[word_index])
            else:
                seq_labels.append(-100)
            prev_word_index = word_index

        aligned_labels.append(seq_labels)

    tokenized_data["labels"] = aligned_labels
    return tokenized_data


**TAKING 20000 LINES FROM TRAINING DATASET**

In [ ]:
newTrainDataset=naamapadam["train"].select(range(len(naamapadam["train"])//49))
newTestDataset=naamapadam["test"].select(range(len(naamapadam["test"])))
newValDataset=naamapadam["validation"].select(range(len(naamapadam["validation"])))

In [27]:
newTrainDataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 20118
})

In [13]:
newTestDataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 867
})

In [14]:
newValDataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 13460
})

**LOADING THE PRETRAINED INDICNER MODEL AND TOKENIZER**

In [9]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels )


2024-03-12 15:20:33.679542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 15:20:33.679662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 15:20:33.820724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
train_dataset = newTrainDataset
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/6 [00:00<?, ?ba/s]

In [11]:
eval_dataset = newValDataset
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

**IMPORTING DATA COLLATOR**

In [12]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

**FUNCTION TO COMPUTE METRICS USING SEQEVAL**

In [20]:
from datasets import load_metric as load_metric_seqeval

# Load the Seqeval metric
seqeval_metric = load_metric_seqeval("seqeval")

def compute_metrics(predictions):
    # Extract predicted labels and true labels
    pred_labels, true_labels = predictions
    pred_labels = np.argmax(pred_labels, axis=2)

    # Exclude special tokens (-100) from predictions and true labels
    filtered_pred_labels = [
        [label_list[pred] for (pred, true) in zip(pred_label, true_label) if true != -100]
        for pred_label, true_label in zip(pred_labels, true_labels)
    ]
    filtered_true_labels = [
        [label_list[true] for (pred, true) in zip(pred_label, true_label) if true != -100]
        for pred_label, true_label in zip(pred_labels, true_labels)
    ]

    # Compute evaluation metrics using Seqeval
    metric_results = seqeval_metric.compute(predictions=filtered_pred_labels, references=filtered_true_labels)

    # Flatten nested dictionaries in metric results
    flattened_results = {}
    for key, value in metric_results.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flattened_results[f"{key}_{sub_key}"] = sub_value
        else:
            flattened_results[key] = value

    return flattened_results


In [15]:
!pip show accelerate

Name: accelerate
Version: 0.27.2
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.10/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: catalyst


**DEFINING ARGUMENTS FOR THE TRAINER**

In [28]:
from transformers import TrainingArguments
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=2e-5
)

In [29]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

**FINE TUNING THE MODEL ON 20000 LINES AND 3 EPOCHS**

In [30]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.151700,0.174329,0.811946,0.861157,0.835828,10213,0.686357,0.698140,0.692199,9786,0.801242,0.842638,0.821419,10568,0.769028,0.802565,0.785439,0.948370
2,0.118400,0.179417,0.802678,0.862724,0.831619,10213,0.672565,0.699162,0.685605,9786,0.812045,0.833176,0.822474,10568,0.764480,0.800144,0.781905,0.947292
3,0.090700,0.195268,0.811352,0.852345,0.831344,10213,0.677690,0.693133,0.685325,9786,0.807854,0.835068,0.821236,10568,0.767892,0.795400,0.781404,0.947059


**SAVING THE INDICNER TOKENIZER AND MODEL AND GETTING ITS FINAL METRICS**

In [36]:
model.save_pretrained('my_indicNER')

In [37]:
tokenizer.save_pretrained('my_indicNER_tokenizer')

('my_indicNER_tokenizer/tokenizer_config.json',
 'my_indicNER_tokenizer/special_tokens_map.json',
 'my_indicNER_tokenizer/vocab.txt',
 'my_indicNER_tokenizer/added_tokens.json',
 'my_indicNER_tokenizer/tokenizer.json')

In [31]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.8313
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.8114
  eval_LOC_recall         =     0.8523
  eval_ORG_f1             =     0.6853
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.6777
  eval_ORG_recall         =     0.6931
  eval_PER_f1             =     0.8212
  eval_PER_number         =      10568
  eval_PER_precision      =     0.8079
  eval_PER_recall         =     0.8351
  eval_loss               =     0.1953
  eval_overall_accuracy   =     0.9471
  eval_overall_f1         =     0.7814
  eval_overall_precision  =     0.7679
  eval_overall_recall     =     0.7954
  eval_runtime            = 0:04:43.81
  eval_samples_per_second =     47.425
  eval_steps_per_second   =      3.953


**TOKENIZING THE TEST SET TO GET ITS OVERALL F1 SCORE**

In [38]:
tokenized_test_set = newTestDataset
tokenized_test_set = tokenized_test_set.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Test dataset",
)

Running tokenizer on Test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
predictions, labels, metrics = trainer.predict(tokenized_test_set)

**MACRO F1 OR OVERALL F1 FOR TEST DATASET**

In [41]:
print(metrics)

{'test_loss': 0.16521050035953522, 'test_LOC_precision': 0.8112903225806452, 'test_LOC_recall': 0.8192182410423453, 'test_LOC_f1': 0.8152350081037278, 'test_LOC_number': 614, 'test_ORG_precision': 0.6510500807754442, 'test_ORG_recall': 0.7676190476190476, 'test_ORG_f1': 0.7045454545454546, 'test_ORG_number': 525, 'test_PER_precision': 0.8410438908659549, 'test_PER_recall': 0.8974683544303798, 'test_PER_f1': 0.8683404776484996, 'test_PER_number': 790, 'test_overall_precision': 0.7756964457252642, 'test_overall_recall': 0.8372213582166926, 'test_overall_f1': 0.8052854649713289, 'test_overall_accuracy': 0.9536017694666465, 'test_runtime': 18.2928, 'test_samples_per_second': 47.396, 'test_steps_per_second': 3.991}


'test_overall_f1': 0.8052854649713289

**TAKING PREDICTIONS ON 25 SENTENCES OF QUESTION 1**

In [43]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [44]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

In [102]:
# import os
# os.remove("/kaggle/working/indicNER_pred.txt")

In [46]:
# Define the absolute path for the output file
output_file_path = "/kaggle/working/indicNER_pred.txt"

#provide the finetuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/my_indicNER_tokenizer")
model = AutoModelForTokenClassification.from_pretrained("/kaggle/working/my_indicNER")

        
# Read the sentences from input.txt
with open("/kaggle/input/sentences-input/input.txt", "r", encoding="utf-8") as file:
    sentences = file.readlines()

# Open the output file for writing
with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate through each sentence
    for sentence in sentences:
        # Remove any leading or trailing whitespace and newline characters
        sentence = sentence.strip()
    
        # Predict labels for the current sentence
        predicted_labels = get_predictions(sentence=sentence, tokenizer=tokenizer, model=model)

        # Write the predicted labels to the output file, separated by spaces
        output_file.write(" ".join(predicted_labels) + "\n")

# Notify the user that the predictions have been saved
print("Predictions saved to indicNER_pred.txt at", output_file_path)

Predictions saved to indicNER_pred.txt at /kaggle/working/indicNER_pred.txt
